In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар? - **Это фиктивный товар.**

Зачем он нужен? - **Он означает, что, если юзер НЕ покупал товары из топ-5000, то он "купил" такой товар).**

Используя этот товар мы смещаем качество рекомендаций. В какую сторону? - **Без этого товара качество рекомендаций будет ухудшаться.**

Можно ли удалить этот товар? - **Да, можно.**

Уберите этот товар и сравните с качеством на семинаре.

In [94]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)  # n_sold - число продаж
# popularity.head()

In [95]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000)['item_id'].tolist()

In [96]:
data_train = data_train.loc[data_train['item_id'].isin(top_5000), :]

In [97]:
# Перевем данные в вид матрицы интеракций
user_item_matrix = pd.pivot_table(data_train, index='user_id', columns='item_id', values='quantity',
                                  aggfunc='count', fill_value=0)

user_item_matrix[user_item_matrix > 0] = 1  # так как в итоге хотим предсказать
user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [98]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [99]:
%%time

model = ItemItemRecommender(K=5, num_threads=4)  # K - кол-во билжайших соседей

model.fit(
    csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
    show_progress=True)

recs = model.recommend(
    userid=userid_to_id[2],  # userid - id от 0 до N
    user_items=csr_matrix(user_item_matrix).tocsr(),  # на вход user-item matrix
    N=5,  # кол-во рекомендаций 
    filter_already_liked_items=False,
    filter_items=None,
    recalculate_user=True)


Wall time: 940 ms


In [100]:
recs

[(3407, 70158.0),
 (2148, 57211.0),
 (3586, 27050.0),
 (3946, 19897.0),
 (2307, 18127.0)]

In [101]:
result = pd.read_csv('predictions_basic.csv')
result = result.loc[:, ['user_id', 'actual', 'itemitem']]
result.head(2)

,user_id,actual,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[999999, 1082185, 981760, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[999999, 1082185, 981760, 1098066, 995242]"


In [102]:
%%time

result['itemitem_only_5000'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in recs])

Wall time: 7.98 ms


In [103]:
result.head(2)

,user_id,actual,itemitem,itemitem_only_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 1098066, 1127831, 995242]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 981760, 1098066, 1127831, 995242]"


In [104]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

print('itemitem', precision_at_k(['itemitem'], result['actual']))
print('itemitem_only_5000', precision_at_k(result['itemitem_only_5000'], result['actual']))

itemitem 0.0
itemitem_only_5000 0.0


**ответ:** Разницы по точности не видно, но, в теории, при удалении всех товаров кроме топ-5000, точность должна возрастать.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [71]:
items_weights = data.groupby(by='item_id')['sales_value'].sum().reset_index().rename(columns={'sales_value':'weight'})
items_weights['weight'] += 1
items_weights['weight'] = np.log(items_weights['weight'])
items_weights['weight'] /= items_weights['weight'].sum()
items_weights.describe()

,item_id,weight
count,8.905100e+04,89051.000000
mean,5.115772e+06,0.000011
std,5.178973e+06,0.000006
min,2.567100e+04,0.000000
25%,9.665830e+05,0.000006
50%,1.448516e+06,0.000010
75%,9.553042e+06,0.000016
max,1.802456e+07,0.000053


In [72]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

In [73]:
result_hw = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_hw.columns=['user_id', 'actual']
result_hw.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


Сделайте предсказания

In [74]:
%%time
result_hw['weighted_random_recommendation'] = result_hw['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result_hw.head(2)

Wall time: 2.33 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[10251782, 12325299, 825075, 925492, 1055480]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[10311753, 889247, 2491085, 876777, 887008]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [105]:
result = pd.read_csv('predictions_basic.csv') 

In [106]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[14111350, 6552528, 984232, 911829, 1084001]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[13986894, 13416416, 51220, 1045214, 1237910]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [118]:
def evaluete_pred(data, true):
    algs = {}
    for col in data.columns[2:]:
        res = precision_at_k(result[col], result[true])
        algs[col] = res
        print(f'{col} : {res}')

In [119]:
evaluete_pred(result, 'actual')

random_recommendation : 0.0
popular_recommendation : 0.0
itemitem : 0.0
cosine : 0.0
tfidf : 0.0
own_purchases : 0.0


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров

In [123]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [128]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [124]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
top_5000[:5]

[6534178, 6533889, 6534166, 6544236, 1404121]

In [125]:
# выборка купившая товары из топ 5000
data_test_top_5k = data_test[data_test['item_id'].isin(top_5000)]
result1 = data_test_top_5k.groupby('user_id')['item_id'].unique().reset_index()
result1.columns=['user_id', 'actual']
result1.head()

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 914190, 95804..."
1,3,"[851057, 872021, 878302, 879948, 909638, 91320..."
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82..."
3,7,"[840386, 889774, 898068, 909714, 953476, 97699..."
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569..."


In [126]:
%%time

items = data_train.item_id.unique()

result1['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result1.head()

Wall time: 298 ms


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[976199, 958569, 6534035, 877455, 905817]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[849870, 857697, 1117219, 1118533, 1101725]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[1112754, 873044, 873902, 1094811, 980823]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[947427, 932006, 1121862, 1100125, 12577242]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[937060, 1121384, 1129049, 879635, 913425]"


In [129]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result1['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result1.head()

Wall time: 66.8 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[976199, 958569, 6534035, 877455, 905817]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[849870, 857697, 1117219, 1118533, 1101725]","[6534178, 6533889, 1029743, 6534166, 1082185]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[1112754, 873044, 873902, 1094811, 980823]","[6534178, 6533889, 1029743, 6534166, 1082185]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[947427, 932006, 1121862, 1100125, 12577242]","[6534178, 6533889, 1029743, 6534166, 1082185]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[937060, 1121384, 1129049, 879635, 913425]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [130]:
%%time

result1['weighted_random_recommendation'] = result1['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result1.head()

Wall time: 2.42 s


,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[976199, 958569, 6534035, 877455, 905817]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1130571, 16769943, 1031300, 1032972, 1052927]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[849870, 857697, 1117219, 1118533, 1101725]","[6534178, 6533889, 1029743, 6534166, 1082185]","[998669, 1289920, 6396252, 8155849, 825249]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[1112754, 873044, 873902, 1094811, 980823]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1792620, 1096351, 5568843, 905217, 12263541]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[947427, 932006, 1121862, 1100125, 12577242]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1116663, 970747, 122761, 1073546, 1075337]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[937060, 1121384, 1129049, 879635, 913425]","[6534178, 6533889, 1029743, 6534166, 1082185]","[13986088, 15863624, 9553177, 903527, 8119156]"


- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [132]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0)

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [134]:
%%time

model = ItemItemRecommender(K=2, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 936 ms


**Ответ:** Лучший показатель для itemitem при K = 2

- Попробуйте стратегии ансамблирования изученных алгоритмов

### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ? 

$\mu$ - средняя оценка по всей таблице

$\bar{r_u}$ - средний рейтинг товаров, которые оценил пользователь $sim$ - функция схожести, (1 - расстояние)

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.